In [6]:
import math
import numpy as np
import os
from Dataset_generation import get_dataset, get_q_batches, get_q_true, get_shuffled_dataset
from Output_data_generation import initialize_folders, get_settings, get_file_name, save_data
from Quantile_procs import get_procs, get_res

In [10]:
from Plot import plot_charts

In [7]:
sgd_lst = ['distro', 'data_size', 'step_size', 'data_sequence']        
tau_vals = [0.1, 0.3, 0.5, 0.9, 0.99]

In [8]:
# Get quantile estimates
def get_q_ests(dataset, tau_lst, method_name, stepsize):
    print ("get_q_ests using method_name {}".format(method_name))
    if len(dataset.shape)>2:
        raise Exception('Dataset for q_est calculation of wrong shape: {}, should be 1d or 2d array'
                        .format(str(dataset.shape)))
    if len(dataset.shape)==1:
        procs = get_procs(dataset, tau_lst, method_name, stepsize=stepsize)
        res = get_res(procs)
    else:
        res = np.zeros((dataset.shape[0], len(tau_lst)))
        procs = np.zeros((dataset.shape[0], len(tau_lst), dataset.shape[1]))
        for idx, dt in enumerate(dataset):
            procs[idx] = get_procs(dt, tau_lst, method_name, stepsize=stepsize)
            res[idx] = get_res(procs[idx])
    return res, procs

In [11]:
def get_normalized_e(true, batches, est):
    upper = est - batches
    bottom = true - batches
    return (upper/bottom)

In [23]:
distro = 'gau_1'
datasize = 1000
tau_lst = tau_vals

dataset = get_dataset(distro, datasize, False)

q_true = get_q_true(distro, tau_vals)
q_batches = get_q_batches(dataset, tau_vals)
sgd_est_res, sgd_res_proc = get_q_ests(dataset, tau_vals, "SGD", 'const')

N_frugal = 20
frugal_res = np.zeros((N_frugal, len(tau_lst)))
frugal_proc = np.zeros((N_frugal, len(tau_lst), datasize))
for i in range(N_frugal):
    frugal_res[i], frugal_proc[i] = get_q_ests(dataset, tau_vals, "Frugal", None)
q_est_res, q_est_proc = frugal_res, frugal_proc
E = get_normalized_e(q_true, sgd_est_res, q_est_res)


get_q_ests using method_name SGD
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal
get_q_ests using method_name Frugal


In [24]:
# file_name = get_file_name(changed_setting, distro, datasize, stepsize, SHUFFLE)
file_name = ('Distribution', distro)

folder_name = "Experiment_results/Frugal_SGD/distro/"

data_dict = {
    'q_true': q_true,
    'q_batches': q_batches,
    'q_est_res': q_est_res,
    'q_est_proc': q_est_proc,
    'E': E
}
    
save_data(folder_name, file_name, tau_vals, data_dict, "Frugal")

Experiment_results/Frugal_SGD/distro/gau_1_q_true.txt
Experiment_results/Frugal_SGD/distro/gau_1_q_batches.txt
Experiment_results/Frugal_SGD/distro/gau_1_q_est_res.txt
Experiment_results/Frugal_SGD/distro/gau_1_q_est_proc.txt
Experiment_results/Frugal_SGD/distro/gau_1_E.txt


In [21]:
plot_charts("Experiment_results/Frugal_SGD/distro")

Experiment_results/Frugal_SGD/distro/gau_1_
q label is  Frugal
q label is  Frugal
q label is  Frugal


In [ ]:
# def sgd_frugal_compare(distro_lst, datasize, tau_lst=tau_vals):
#     #distro changes, use the biggest datasize, do not shuffle
#     for distro in distro_lst:
#         q_true = get_q_true(distro, tau_lst)
#         dataset = get_dataset(distro, datasize, False)
#         q_batches = get_q_batches(dataset, tau_lst)
        
#         sgd_res, sgd_proc = get_q_ests(dataset, 'const', tau_lst)
        
#         N_frugal = 20
#         frugal_res = np.zeros((N_frugal, len(tau_lst)))
#         frugal_proc = np.zeros((N_frugal, len(tau_lst), datasize))
#         for i in range(N_frugal):
#             frugal_res[i], frugal_proc[i] = get_q_ests(dataset, 'frugal', tau_lst)
        
#         ax_name = 'Tested on '+distro+' distritbution with '+str(datasize)+' data points'
#         fig, lgd = plot_procs(ax_name, tau_vals, q_true, frugal_proc, sgd_proc)
#         title = fig.suptitle('Quantile Estimation: Frugal vs SGD')

#         fd = "Experiment_results/Frugal_SGD/"
#         plt.savefig(fd+distro+'.png', bbox_extra_artists=(lgd, title), bbox_inches='tight')
        
        
# sgd_frugal_compare(distros, 1000)